## Загрузка данных

In [24]:
# imports
import pandas as pd
import numpy as np
import warnings

from scipy.stats import randint, uniform, loguniform

from sklearn.metrics import mean_absolute_percentage_error, make_scorer

# Импорт бустинг моделей
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV
from optuna.integration import OptunaSearchCV
from optuna.distributions import (
    IntDistribution,
    FloatDistribution,
    CategoricalDistribution
)

warnings.filterwarnings("ignore")
RANDOM_STATE = 42
#TODO

{'loss_function': 'RMSE'}


In [ ]:
model = LGBMRegressor(device='gpu')
try:
    model.fit([[0]], [0])  # Пробное обучение
    print("GPU поддерживается в LightGBM")
except Exception as e:
    print("Ошибка:", e)

#TODO проверить, поддерживается ли GPU

In [7]:
df = pd.read_csv('data/train_df_processed.csv')
test_df = pd.read_csv('data/test_df_processed.csv')
#TODO

In [8]:
df = df.drop(['negative_review', 'positive_review'], axis=1)
test_df = test_df.drop(['negative_review', 'positive_review'], axis=1)
#TODO

In [9]:
# Приводим столбцы в тренировочном датасете к нужному типу данных
df['hotel'] = df['hotel'].astype('category')
df['nationality'] = df['nationality'].astype('category')
# df['negative_review'] = df['negative_review'].astype('category')
# df['positive_review'] = df['positive_review'].astype('category')
df['room_type'] = df['room_type'].astype('category')
df['neg_words_supremacy'] = df['neg_words_supremacy'].astype('category')
df['negative_words_level'] = df['negative_words_level'].astype('category')
df['positive_words_level'] = df['positive_words_level'].astype('category')

# Приводим столбцы в тестовом датасете к нужному типу данных
test_df['hotel'] = test_df['hotel'].astype('category')
test_df['nationality'] = test_df['nationality'].astype('category')
# test_df['negative_review'] = test_df['negative_review'].astype('category')
# test_df['positive_review'] = test_df['positive_review'].astype('category')
test_df['room_type'] = test_df['room_type'].astype('category')
test_df['neg_words_supremacy'] = test_df['neg_words_supremacy'].astype('category')
test_df['negative_words_level'] = test_df['negative_words_level'].astype('category')
test_df['positive_words_level'] = test_df['positive_words_level'].astype('category')
#TODO

In [48]:
df

,hotel,nationality,negative_words,positive_words,score,room_type,neg_words_supremacy,negative_words_level,positive_words_level,neg_emb_0,...,pos_emb_758,pos_emb_759,pos_emb_760,pos_emb_761,pos_emb_762,pos_emb_763,pos_emb_764,pos_emb_765,pos_emb_766,pos_emb_767
0,DoubleTree by Hilton Hotel Amsterdam NDSM Wharf,United Kingdom,72,8,5.4,Queen Guest Room,1,high,low,-0.048846,...,0.008750,-0.012482,0.054320,0.006842,-0.003336,-0.003842,-0.033210,0.033710,0.005292,-0.000549
1,Savoy Hotel Amsterdam,Malaysia,246,41,9.6,Small Double Room,1,high,medium,0.017662,...,-0.023608,-0.005871,0.059574,0.025843,-0.036666,-0.008732,-0.020834,0.007470,0.049503,-0.013883
2,Park Plaza London Riverbank,United Kingdom,19,19,8.8,Superior Double Room,0,low,low,-0.013490,...,-0.031864,0.028069,0.040875,0.038138,-0.003895,0.021191,0.003557,0.005436,0.037507,-0.016811
3,Palais Hansen Kempinski Vienna,Oman,14,5,10.0,Deluxe Room,1,low,low,0.032819,...,-0.024973,0.110654,0.064530,-0.004439,0.015669,-0.006296,0.028076,-0.034253,0.042687,-0.014665
4,Catalonia Rigoletto,United Kingdom,3,11,9.2,Double or Twin Room,0,low,low,0.011261,...,-0.021859,0.044477,-0.015735,-0.020282,-0.026186,-0.026984,-0.023102,-0.003747,0.035859,-0.049596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412231,Club Quarters Hotel Gracechurch,Belgium,24,75,9.2,Small Queen Room,0,medium,high,-0.024681,...,-0.035830,-0.010778,-0.000357,-0.016725,-0.027403,-0.041550,-0.013371,-0.001772,0.029651,-0.015170
412232,Park Plaza Westminster Bridge London,United Kingdom,4,17,10.0,Studio Triple,0,low,low,0.008482,...,-0.031002,0.008744,0.095677,0.022748,-0.037872,-0.017063,-0.056433,-0.024916,0.011959,-0.035305
412233,Park Lane Mews Hotel,United Kingdom,21,6,7.9,Double Room,1,medium,low,0.029697,...,-0.014793,-0.012526,0.033770,-0.000607,-0.033183,-0.035154,-0.033073,0.027250,0.048244,-0.011523
412234,Apex City Of London Hotel,United Kingdom,29,41,8.3,City King or Twin Room,0,medium,medium,-0.056332,...,-0.020531,-0.031567,0.068144,0.037584,-0.038096,-0.006007,-0.055387,0.015724,0.049305,0.008255


## Подготовка данных

In [11]:
num_cols = ['negative_words', 'positive_words']
cat_cols = ['neg_words_supremacy', 'negative_words_level', 'positive_words_level']
label_cols = ['hotel', 'nationality', 'room_type']

catboost_col_names = cat_cols + label_cols
#TODO

In [12]:
# Функция для метрики MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100


mape_custom_score = make_scorer(mape, greater_is_better=False)
#TODO

In [13]:
X_train = df.drop('score', axis=1)
y_train = df['score']
#TODO

In [14]:
predictions = pd.DataFrame({
    'id': range(len(test_df)),
})
predictions
#TODO

,id
0,0
1,1
2,2
3,3
4,4
...,...
103143,103143
103144,103144
103145,103145
103146,103146


## Обучение CatBoost

### CatBoost OptunaSearchCV

In [28]:
# Расширенная сетка гиперпараметров
catboost_optuna_params = {
    # Основные параметры
    'iterations': IntDistribution(50, 1000, step=50),  # Увеличенный диапазон
    'learning_rate': FloatDistribution(0.001, 0.3, log=True),  # Более широкий диапазон
    'depth': IntDistribution(4, 12),  # Расширенная глубина деревьев

    # Регуляризация
    'l2_leaf_reg': FloatDistribution(1e-8, 100, log=True),  # Более широкий диапазон
    'random_strength': FloatDistribution(1e-8, 10, log=True),  # Сила случайности

    # Управление бэггингом
    'bagging_temperature': FloatDistribution(0.0, 1.0),  # Температура бэггинга
    'subsample': FloatDistribution(0.5, 1.0),  # Аналогично subsample в LightGBM
    'colsample_bylevel': FloatDistribution(0.5, 1.0),  # Аналог colsample_bytree

    # Дополнительные параметры
    'min_data_in_leaf': IntDistribution(1, 20),  # Минимальное количество объектов в листе
    'max_bin': IntDistribution(64, 512),  # Количество бинов для числовых признаков
    'one_hot_max_size': IntDistribution(2, 16),  # Размер для one-hot кодирования категориальных признаков

    # Стратегии роста деревьев
    'grow_policy': CategoricalDistribution(['SymmetricTree', 'Depthwise', 'Lossguide']),

    # Параметры оценки листьев
    'leaf_estimation_iterations': IntDistribution(1, 10),  # Итерации для оценки листьев
    'leaf_estimation_method': CategoricalDistribution(['Newton', 'Gradient']),  # Метод оптимизации

    # Автоматическая балансировка (для классификации, но можно тестировать)
    # 'auto_class_weights': CategoricalDistribution(['None', 'Balanced', 'SqrtBalanced']),

    # Управление категориальными признаками
    'ctr_leaf_count_limit': IntDistribution(10, 1000),  # Ограничение на количество категорий
    'has_time': CategoricalDistribution([True, False]),  # Учет временного фактора

    # Ранняя остановка
    'early_stopping_rounds': IntDistribution(10, 100)  # Добавлена ранняя остановка
}

In [17]:
# Используем OptunaSearchCV
catboost_optuna = OptunaSearchCV(
    # estimator=CatBoostRegressor(),
    estimator=CatBoostRegressor(
        random_state=RANDOM_STATE,
        verbose=False,  # Отключаем вывод логов CatBoost
        thread_count=-1,  # Используем все доступные ядра
    ),
    param_distributions=catboost_optuna_params,
    scoring=mape_custom_score,  # Метрика для оптимизации
    n_trials=15,  # Количество испытаний
    cv=5,  # Кросс-валидация
    n_jobs=-1,  # Использование всех ядер
    verbose=0,
)

/var/folders/d5/3r484lmx4rx7jdlt6zl0_bkc0000gn/T/ipykernel_46475/2563794217.py:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  catboost_optuna = OptunaSearchCV(


In [ ]:
# Запускаем поиск гиперпараметров
catboost_optuna.fit(X_train, y_train, cat_features=catboost_col_names)

[I 2025-05-09 00:12:12,398] A new study created in memory with name: no-name-72cadddb-86d9-4408-9bc1-637d0934fc06


In [ ]:
# Выводим лучшую оценку
catboost_optuna.best_score_

In [ ]:
# Предсказываем таргет
predictions['catboost_optuna_pred'] = catboost_optuna.predict(test_df)

### CatBoost RandomSearchCV

In [25]:
# Сетка гиперпараметров для CatBoost с RandomizedSearchCV
catboost_rs_params = {
    # Основные параметры
    'iterations': randint(50, 1000),  # Количество итераций
    'learning_rate': loguniform(0.001, 0.3),  # Скорость обучения
    'depth': randint(4, 12),  # Глубина деревьев

    # Регуляризация
    'l2_leaf_reg': loguniform(1e-8, 100),  # L2-регуляризация
    'random_strength': loguniform(1e-8, 10),  # Сила случайности

    # Управление бэггингом
    'bagging_temperature': uniform(0.0, 1.0),  # Температура бэггинга
    'subsample': uniform(0.5, 0.5),  # Доля данных для обучения
    'colsample_bylevel': uniform(0.5, 0.5),  # Доля признаков на уровень

    # Дополнительные параметры
    'min_data_in_leaf': randint(1, 20),  # Мин. объектов в листе
    'max_bin': randint(64, 512),  # Число бинов
    'one_hot_max_size': randint(2, 16),  # Размер one-hot кодирования

    # Стратегии роста деревьев
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],  # Стратегия роста

    # Параметры оценки листьев
    'leaf_estimation_iterations': randint(1, 10),  # Итерации оценки листьев
    'leaf_estimation_method': ['Newton', 'Gradient'],  # Метод оценки

    # Управление категориальными признаками
    'ctr_leaf_count_limit': randint(10, 1000),  # Ограничение на количество категорий
    'has_time': [True, False],  # Учет временного фактора

    # Ранняя остановка
    'early_stopping_rounds': randint(10, 100)  # Количество итераций для остановки
}

In [26]:
# Используем OptunaSearchCV
catboost_rs = RandomizedSearchCV(
    # estimator=CatBoostRegressor(),
    estimator=CatBoostRegressor(
        random_state=RANDOM_STATE,
        verbose=False,  # Отключаем вывод логов CatBoost
        thread_count=-1,  # Используем все доступные ядра
        task_type='GPU',
    ),
    param_distributions=catboost_rs_params,
    scoring=mape_custom_score,  # Метрика для оптимизации
    n_iter=15,  # Количество испытаний
    cv=5,  # Кросс-валидация
    n_jobs=1,  # Использование всех ядер
    verbose=0,
)

In [27]:
from catboost import CatBoostRegressor
print(CatBoostRegressor().get_params().get('task_type', 'No GPU support'))
# Должно вернуть 'GPU'

No GPU support


In [21]:
# Запускаем поиск гиперпараметров
catboost_rs.fit(X_train, y_train, cat_features=catboost_col_names)

KeyboardInterrupt: 

In [ ]:
# Выводим лучшую оценку
catboost_rs.best_score_

In [ ]:
# Предсказываем таргет
predictions['catboost_rs_pred'] = catboost_rs.predict(test_df)

## Обучение LightGBM

### LGBM OptunaSearchCV

In [29]:
# Расширенная сетка гиперпараметров для LightGBM
lgbm_optuna_params = {
    # Основные параметры
    'num_leaves': IntDistribution(20, 400),  # Количество листьев в дереве
    'max_depth': IntDistribution(3, 10),  # Максимальная глубина деревьев
    'min_child_samples': IntDistribution(5, 200),  # Мин. количество объектов в листе
    'min_child_weight': FloatDistribution(1e-3, 10),  # Вес для ограничения разделения узла

    # Скорость обучения
    'learning_rate': FloatDistribution(0.005, 0.3, log=True),  # Широкий диапазон

    # Регуляризация
    'lambda_l1': FloatDistribution(1e-5, 10, log=True),  # L1-регуляризация
    'lambda_l2': FloatDistribution(1e-5, 10, log=True),  # L2-регуляризация
    'min_gain_to_split': FloatDistribution(1e-3, 10),  # Мин. улучшение для разделения

    # Подвыборка
    'subsample': FloatDistribution(0.4, 1.0),  # Доля данных для обучения
    'subsample_freq': IntDistribution(0, 5),  # Частота бэггинга
    'colsample_bytree': FloatDistribution(0.4, 1.0),  # Доля признаков на дерево

    # Число деревьев
    'n_estimators': IntDistribution(50, 1000),  # Количество итераций

    # Дополнительные параметры
    'max_bin': IntDistribution(64, 1024),  # Количество бинов для числовых признаков
    'cat_smooth': FloatDistribution(1.0, 100.0),  # Гладкость для категориальных признаков
    'cat_l2': FloatDistribution(1e-8, 10),  # L2-регуляризация для категориальных

    # Стратегии построения деревьев
    'boosting_type': CategoricalDistribution(['gbdt', 'dart', 'goss']),  # Тип бустинга

    # Другие параметры
    # 'early_stopping_rounds': IntDistribution(10, 50),  # Ранняя остановка
    'feature_fraction_bynode': FloatDistribution(0.1, 1.0)  # Доля признаков на узел
}
#TODO

In [41]:
# Настройка Optuna
lgbm_optuna = OptunaSearchCV(
    estimator=LGBMRegressor(
        objective='regression',
        metric='mape',
        boosting_type='gbdt',
        random_state=RANDOM_STATE,
        verbosity=-1,
        device='gpu',  # Включаем GPU
        tree_learner='feature_parallel'  # Опционально: параллелизм на GPU
    ),
    param_distributions=lgbm_optuna_params,
    scoring=mape_custom_score,  # Для регрессии
    n_trials=40,  # Количество испытаний #TODO Увеличить|уменьшить, если что
    cv=5,  # Кросс-валидация
    n_jobs=1,  # Параллельные вычисления
    verbose=0
)
#TODO

In [42]:
# Обучаем модель
lgbm_optuna.fit(X_train, y_train)
#TODO

[I 2025-05-11 18:38:51,257] A new study created in memory with name: no-name-27c2d759-f1d0-4b70-b74b-7bf97ba2bbce
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[W 2025-05-11 18:39:57,150] Trial 0 failed with parameters: {'num_leaves': 234, 'max_depth': 7, 'min_child_samples': 7, 'min_child_weight': 5.6874704444966575, 'learning_rate': 0.012161950671868, 'lambda_l1': 0.7752326559848562, 'lambda_l2': 0.0003524574109819891, 'min_gain_to_split': 1.0072895467334158, 'subsample': 0.92732189415995, 'subsample_freq': 1, 'colsample_bytree': 0.45032437638856804, 'n_estimators': 252, 'max_bin': 243, 'cat_smooth': 51.019781945587376, 'cat_l2': 2.386064327765593, 'boosting_type': 'goss', 'feature_fraction_bynode': 0.34613496701325214} because of the following error: The value nan is not acceptable.
[W 2025-05-11 18:39

OptunaSearchCV(cv=5,
               estimator=LGBMRegressor(metric='mape', objective='regression',
                                       random_state=42, verbosity=-1),
               n_jobs=1, n_trials=15,
               param_distributions={'boosting_type': CategoricalDistribution(choices=('gbdt', 'dart', 'goss')),
                                    'cat_l2': FloatDistribution(high=10.0, log=False, low=1e-08, step=None),
                                    'cat_smooth': FloatDistribution(high=100.0, log=False, low=1...
                                    'min_gain_to_split': FloatDistribution(high=10.0, log=False, low=0.001, step=None),
                                    'n_estimators': IntDistribution(high=1000, log=False, low=50, step=1),
                                    'num_leaves': IntDistribution(high=400, log=False, low=20, step=1),
                                    'subsample': FloatDistribution(high=1.0, log=False, low=0.4, step=None),
                                    'subsample_freq': IntDistribution(high=5, log=False, low=0, step=1)},
               scoring=make_scorer(mape, greater_is_better=False, response_method='predict'))

In [43]:
# Выводим лучшую оценку
lgbm_optuna.best_score_
#TODO

-10.543868275488705

In [44]:
# Предсказываем таргет
predictions['lgbm_optuna_pred'] = lgbm_optuna.predict(test_df)
#TODO

### LGBM RandomSearchCV

In [10]:
# Расширенная сетка гиперпараметров для RandomizedSearchCV
lgbm_rs_params = {
    # Основные параметры
    'num_leaves': randint(20, 400),  # Количество листьев в дереве
    'max_depth': randint(3, 10),  # Максимальная глубина деревьев
    'min_child_samples': randint(5, 200),  # Мин. количество объектов в листе
    'min_child_weight': loguniform(1e-3, 10),  # Вес для ограничения разделения узла

    # Скорость обучения
    'learning_rate': loguniform(0.005, 0.3),  # Широкий диапазон

    # Регуляризация
    'reg_alpha': loguniform(1e-5, 10),  # L1-регуляризация (аналог lambda_l1)
    'reg_lambda': loguniform(1e-5, 10),  # L2-регуляризация (аналог lambda_l2)
    'min_gain_to_split': loguniform(1e-3, 10),  # Мин. улучшение для разделения

    # Подвыборка
    'subsample': uniform(0.4, 0.6),  # Доля данных для обучения
    'subsample_freq': randint(0, 5),  # Частота бэггинга
    'colsample_bytree': uniform(0.4, 0.6),  # Доля признаков на дерево

    # Число деревьев
    'n_estimators': randint(50, 1000),  # Количество итераций

    # Дополнительные параметры
    'max_bin': randint(64, 1024),  # Количество бинов для числовых признаков
    'cat_smooth': loguniform(1.0, 100.0),  # Гладкость для категориальных признаков
    'cat_l2': loguniform(1e-8, 10),  # L2-регуляризация для категориальных

    # Стратегии построения деревьев
    'boosting_type': ['gbdt', 'dart', 'goss'],  # Тип бустинга

    # Другие параметры
    # 'early_stopping_rounds': randint(10, 50),  # Ранняя остановка
    'feature_fraction_bynode': loguniform(0.1, 1.0)  # Доля признаков на узел
}

In [14]:
# Настройка Optuna
lgbm_rs = RandomizedSearchCV(
    estimator=LGBMRegressor(
        objective='regression',
        metric='mape',
        boosting_type='gbdt',
        random_state=RANDOM_STATE,
        verbosity=-1,
    ),
    param_distributions=lgbm_rs_params,
    scoring=mape_custom_score,  # Для регрессии
    n_iter=15,  # Количество испытаний
    cv=None,  # Кросс-валидация
    n_jobs=1,  # Параллельные вычисления
    verbose=0
)

In [15]:
# Обучаем модель
lgbm_rs.fit(X_train, y_train)

[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
Exception ignored in: <function ResourceTracker.__del__ at 0x10707f740>
Traceback (most recent call last):
  File "/Users/moncervers/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/Users/moncervers/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/Users/moncervers/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
/

RandomizedSearchCV(estimator=LGBMRegressor(metric='mape',
                                           objective='regression',
                                           random_state=42, verbosity=-1),
                   n_iter=15, n_jobs=4,
                   param_distributions={'boosting_type': ['gbdt', 'dart',
                                                          'goss'],
                                        'cat_l2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x33a4e02f0>,
                                        'cat_smooth': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at...
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x166830e30>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x166830fb0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1668320c0>,
                                        'subsample_freq': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1668329f0>},
                   scoring=make_scorer(mape, greater_is_better=False, response_method='predict'))

In [16]:
# Выводим лучшую оценку
lgbm_rs.best_score_

np.float64(-10.834411807666246)

In [17]:
# Предсказываем таргет
predictions['lgbm_rs_pred'] = lgbm_rs.predict(test_df)

## Сохранение submission

In [49]:
# predictions['lgbm_optuna_pred'] = predictions['lgbm_optuna_pred'].apply(lambda x: round(x, 1))
predictions
#TODO Проверить, корректно ли все выводится

,id,lgbm_optuna_pred
0,0,9.4
1,1,8.8
2,2,8.2
3,3,9.9
4,4,8.7
...,...,...
103143,103143,9.6
103144,103144,9.8
103145,103145,9.0
103146,103146,9.9


In [50]:
submission = predictions[['id', 'lgbm_optuna_pred']]
submission['lgbm_optuna_pred'] = submission['lgbm_optuna_pred'].apply(lambda x: 10 if x > 10 else x)
submission
#TODO

,id,lgbm_optuna_pred
0,0,9.4
1,1,8.8
2,2,8.2
3,3,9.9
4,4,8.7
...,...,...
103143,103143,9.6
103144,103144,9.8
103145,103145,9.0
103146,103146,9.9


In [51]:
submission.to_csv('submission.csv', index=False)
#TODO